In [1]:
import pymysql as mariadb

try:
    conn = mariadb.connect(
     host='db',
     user='root',
     passwd='1234',
     db='rental',
     charset='utf8mb4'
    )
    
    cursor = conn.cursor()
    sql = """
        CREATE TABLE `rentInfo` (
  `ID` INT NOT NULL AUTO_INCREMENT,
  `url` VARCHAR(100) NULL,
  `update_date` VARCHAR(50) NULL,
  `title` VARCHAR(100) NULL,
  `address` VARCHAR(200) NULL,
  `label` CHAR(5) NULL,
  `temp` CHAR(1) NULL,
  `sex` CHAR(1) NULL,
  `smoke` CHAR(1) NULL,
  `pet` CHAR(1) NULL,
  `cook` CHAR(1) NULL,
  `space` DECIMAL(9,2) NULL,
  `rent` INT NULL,
  `floor` INT NULL,
  `stories` INT NULL,
  `pattern` VARCHAR(20) NULL,
  `landlord` VARCHAR(100) NULL,
  `description` TEXT(2000) NULL,
  `lat` DECIMAL(20,17) NULL,
  `lng` DECIMAL(20,17) NULL,
  `cityID` INT NULL,
  PRIMARY KEY (`ID`),
  UNIQUE INDEX `url_UNIQUE` (`url` ASC))
  ENGINE = InnoDB DEFAULT  
  CHARACTER SET = utf8mb4 COLLATE = utf8mb4_general_ci;
        """
    try:
        cursor.execute(sql)
    except Exception as e:
        print('Execute Failed!\nError Code is %s;\nError Content is %s;' % (e.args[0],e.args[1]))
        cursor.close()
        conn.close()
except Exception as e:
    print('Connection Failed!\nError Code is %s;\nError Content is %s;' % (e.args[0],e.args[1]))

In [2]:
# !pip install pymysql

In [3]:
# import redis

# r = redis.Redis(
#     host='redis',)

In [4]:
# import glob
# from multiprocessing import Process, Queue
# import json

# def read_file(q1):
#     fil = glob.glob('/home/johndoe/Desktop/test/rent_data/*')
#     for filLis in fil:
#         for detail in glob.glob('{}/*'.format(filLis)):
#             q1.put(detail)

# if __name__ == "__main__":

#     q1=Queue()
#     read_file(q1)
#     print(q1.qsize())
#     q1.close()
    

In [5]:
import glob
from multiprocessing import Process, Queue
import json
q1=Queue()
fil = glob.glob('rent_data/*')
for filLis in fil:
    for detail in glob.glob('{}/*'.format(filLis)):
        with open('{}'.format(detail),encoding='utf-8') as f:
            jSon = json.load(f)
            #print(jSon)
            q1.put(jSon)
#print(q1.qsize())




In [6]:
print(q1.qsize())

24805


In [8]:
def write(lock,obj):
    obj = obj
    lock.acquire()
    try:
        conn = mariadb.connect(
            host='db',
            user='root',
            passwd='1234',
            db='rental',
            charset='utf8mb4'
        )
        cursor = conn.cursor()
        sql = '''INSERT INTO rentInfo (url,update_date,title,address,label,temp,sex,pet,cook,smoke,space,floor,stories,
                pattern,rent,landlord,description,lat,lng,cityID) VALUES 
                (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'''
        values = (obj['url'], obj['update'],obj['title'], obj['address'],obj['label'], obj['temp'],
                obj['sex'], obj['pet'], obj['cook'], obj['smoke'],obj['space'], obj['floor'], obj['stories'],
                obj['pattern'], obj['rent'],obj['landlord'], obj['description'], obj['lat'],obj['lng'], obj['cityID'])
        try:
            cursor.execute(sql,values)
            conn.commit()
        except Exception as e:
            print(obj)
            conn.rollback()
            print('Execute Failed!\nError Code is %s;\nError Content is %s;' % (e.args[0],e.args[1]))

        cursor.close()

        conn.close()
    except Exception as e:
        print(obj)
        print('Connection Failed!\nError Code is %s;\nError Content is %s;' % (e.args[0],e.args[1]))
    finally:
        lock.release()

In [9]:
import pymysql as mariadb
import time
from multiprocessing import Process, Lock,
lock = Lock()
while not q1.empty():
    try:
        obj = q1.get()
        p = Process(target=write, args=(lock, obj)) 
        p.start()
        p.join(timeout=0.1)
    except Exception as e:
        print(e)
        print(obj)

{'pattern': 4, 'cook': 'Y', 'sex': '限女生', 'pet': 'N', 'address': '台北市大安區復興南路二段', 'stories': 4, 'floor': 3, 'space': 10.0, 'rent': 17000, 'title': '科技捷運公園套房', 'landlord': 'NA', 'smoke': 'N', 'temp': 'N', 'url': 'http://rent.yungching.com.tw/house/1279286/', 'label': '住', 'description': '房屋描述科技捷運公園套房近大安科技大樓捷運站環境清幽', 'update': '2017-08-16', 'lat': 25.0272712, 'lng': 121.5435514, 'cityID': 106}
Execute Failed!
Error Code is 1406;
Error Content is Data too long for column 'sex' at row 1;
{'pattern': 5, 'cook': 'Y', 'sex': '限男生', 'pet': 'N', 'address': '台北市大安區溫州街', 'stories': 5, 'floor': 1, 'space': 15.0, 'rent': 19000, 'title': '套房1加1', 'landlord': 'NA', 'smoke': 'N', 'temp': 'N', 'url': 'http://rent.yungching.com.tw/house/1279278/', 'label': '住', 'description': '房屋描述套房1加1一樓文化靜巷生活機能佳', 'update': '2017-08-16', 'lat': 25.022605, 'lng': 121.5332326, 'cityID': 106}
Execute Failed!
Error Code is 1406;
Error Content is Data too long for column 'sex' at row 1;
{'pattern': 5, 'cook': 'N', 'sex': '限

/opt/conda/lib/python3.6/site-packages/pymysql/cursors.py:166: Warning: (1265, "Data truncated for column 'space' at row 1")
  result = self._query(query)
/opt/conda/lib/python3.6/site-packages/pymysql/cursors.py:166: Warning: (1265, "Data truncated for column 'space' at row 1")
  result = self._query(query)
/opt/conda/lib/python3.6/site-packages/pymysql/cursors.py:166: Warning: (1265, "Data truncated for column 'space' at row 1")
  result = self._query(query)
/opt/conda/lib/python3.6/site-packages/pymysql/cursors.py:166: Warning: (1265, "Data truncated for column 'space' at row 1")
  result = self._query(query)
/opt/conda/lib/python3.6/site-packages/pymysql/cursors.py:166: Warning: (1265, "Data truncated for column 'space' at row 1")
  result = self._query(query)
/opt/conda/lib/python3.6/site-packages/pymysql/cursors.py:166: Warning: (1265, "Data truncated for column 'space' at row 1")
  result = self._query(query)
/opt/conda/lib/python3.6/site-packages/pymysql/cursors.py:166: Warning

/opt/conda/lib/python3.6/site-packages/pymysql/cursors.py:166: Warning: (1265, "Data truncated for column 'space' at row 1")
  result = self._query(query)
/opt/conda/lib/python3.6/site-packages/pymysql/cursors.py:166: Warning: (1265, "Data truncated for column 'space' at row 1")
  result = self._query(query)
/opt/conda/lib/python3.6/site-packages/pymysql/cursors.py:166: Warning: (1265, "Data truncated for column 'space' at row 1")
  result = self._query(query)
/opt/conda/lib/python3.6/site-packages/pymysql/cursors.py:166: Warning: (1265, "Data truncated for column 'space' at row 1")
  result = self._query(query)
/opt/conda/lib/python3.6/site-packages/pymysql/cursors.py:166: Warning: (1265, "Data truncated for column 'space' at row 1")
  result = self._query(query)
/opt/conda/lib/python3.6/site-packages/pymysql/cursors.py:166: Warning: (1265, "Data truncated for column 'space' at row 1")
  result = self._query(query)
/opt/conda/lib/python3.6/site-packages/pymysql/cursors.py:166: Warning

/opt/conda/lib/python3.6/site-packages/pymysql/cursors.py:166: Warning: (1265, "Data truncated for column 'space' at row 1")
  result = self._query(query)
/opt/conda/lib/python3.6/site-packages/pymysql/cursors.py:166: Warning: (1265, "Data truncated for column 'space' at row 1")
  result = self._query(query)
/opt/conda/lib/python3.6/site-packages/pymysql/cursors.py:166: Warning: (1265, "Data truncated for column 'space' at row 1")
  result = self._query(query)
/opt/conda/lib/python3.6/site-packages/pymysql/cursors.py:166: Warning: (1265, "Data truncated for column 'space' at row 1")
  result = self._query(query)
/opt/conda/lib/python3.6/site-packages/pymysql/cursors.py:166: Warning: (1265, "Data truncated for column 'space' at row 1")
  result = self._query(query)
/opt/conda/lib/python3.6/site-packages/pymysql/cursors.py:166: Warning: (1265, "Data truncated for column 'space' at row 1")
  result = self._query(query)
/opt/conda/lib/python3.6/site-packages/pymysql/cursors.py:166: Warning

In [ ]:
# in/out all string , utf8mb4 let emoji acceptable, int float still insert if out of range

In [ ]:
# try:
#     conn = mariadb.connect(
#             host='db',
#             user='root',
#             passwd='1234',
#             db='rental',
#             charset='utf8'
#         )
#     cursor = conn.cursor()
#     sql = '''INSERT INTO rent (url,update_date,title,address,label,temp,sex,pet,cook,smoke,space,floor,stories,
#                     pattern,rent,landlord,description,lat,lng,cityID) VALUES\
#                     (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'''
#     value = 
#     try:
#         cursor.execute(sql,())
#         conn.commit()
#     except Exception as e:
#         conn.rollback()
#         print('Execute Failed!\nError Code is %s;\nError Content is %s;' % (e.args[0],e.args[1]))

#     cursor.close()

#     conn.close()
# except Exception as e:
#         print('Connection Failed!\nError Code is %s;\nError Content is %s;' % (e.args[0],e.args[1]))